# 0.라이브러리 로딩

In [1]:
import os
import librosa
import torch
from logger import utils
from tqdm import tqdm
from glob import glob
from pydub import AudioSegment
from logger.utils import traverse_dir

# Cuda setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# configure loading
args = utils.load_config('./configs/sins.yaml')

# set path
MP4_DATA_PATH   = 'preprocess/mp4'
ORIGINAL_PATH   = 'preprocess/original/'
DEMUCS_PATH     = 'preprocess/demucs/'
NORM_PATH       = 'preprocess/norm/'
TEMP_LOG_PATH   = 'temp_ffmpeg_log.txt'  # ffmpeg의 무음 감지 로그의 임시 저장 위치

# 1.데이터 전처리
***
1. 난 전처리가 필요없다. 배경음 제거가 완벽하고, 모든 데이터들도 특정 길이로 잘 잘려져있다.
    - 데이터를 전부 data/train/audio/안에 다 집어 넣고
        ```
        # training dataset
        data/train/audio/aaa.wav
        data/train/audio/bbb.wav
        ...
        ```
    - 1.6 validation 분리단계로 점프
***
2. 난 거의 다 되어 있지만 데이터가 너무 길다. 특정 길이로 자르고 싶다.
    - 데이터를 전부 preprocess/split 안에 다 집어 넣고
        ```
        # training dataset
        preprocess/norm/aaa.wav
        preprocess/norm/bbb.wav
        ...
        ```
    - 1.4 split 단계로 점프
***
3. 난 배경음도 제거해야되고 데이터도 길다. 거의 날 것의 상태다.
    - 데이터를 전부 preprocess/original 안에 다 집어넣고
        ```
        # training dataset
        preprocess/original/aaa.wav
        preprocess/original/bbb.wav
        ...
        ```
    - 1.2부터 demucs 단계로 점프
***
4. 난 아무것도 안되어 있고, 심지어 mp4파일이다.
    - 데이터들 전부 preprocess/mp4에 다 집어넣고
        ```
        # training dataset
        preprocess/mp4/aaa.mp4
        preprocess/mp4/bbb.mp4
        ...
        ```
    - 1.1부터 차례대로 진행

## 1.1데이터가 mp4인 경우 (wav만 있는 경우에는 패스)
preprocess/mp4 안에 있는 mp4파일을 wav로 변경 해서 preprocess/original 에 저장

In [12]:
def mp4_to_wav(input_dir : str, input_file: str, output_dir: str):
    """mp4파일을 wav형식으로 변환합니다.
    Args:
        input_dir (str) : 입력 mp4파일의 path
        input_file (str) : 입력 mp4파일의 이름
        output_dir (str) : 출력 wav파일의 path
    """
    ext = os.path.splitext(input_file)[1][1:]

    if ext != "mp4":
        return 
    else :
        track = AudioSegment.from_file(os.path.join(input_dir,input_file),  format= 'mp4')
        track = track.set_frame_rate(44100)
        track.export(os.path.join(output_dir, input_file[:-4]+".wav"), format='wav')


filelist =  traverse_dir(
    MP4_DATA_PATH,
    extension='mp4',
    is_pure=True,
    is_sort=True,
    is_ext=True)

for file in tqdm(filelist):
    mp4_to_wav(MP4_DATA_PATH, file, ORIGINAL_PATH)

0it [00:00, ?it/s]


## 1.2 무음제거 (Demucs)
preprocess/original에 있는 wav파일들의 음악소리를 제거하고 목소리만 추출해서 preprocess/demucs 에 저장

In [13]:
from sep_wav import demucs

demucs(ORIGINAL_PATH, DEMUCS_PATH)

Sample rate: 44100


목소리 추출 중...: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:35<00:00, 35.42s/it]


## 1.3 normalize
preprocess/demucs에 있는 배경음이 제거된 데이터들을 노멀라이즈 (sample rate값을 바꿀 수 있음) 해서 preprocess/norm에 저장

In [14]:
from sep_wav import audio_norm

for filepath in tqdm(glob(DEMUCS_PATH+"*.wav"), desc="노멀라이징 작업 중..."):
    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(NORM_PATH, filename) + ".wav"
    audio_norm(filepath, out_filepath, sample_rate = 44100)

노멀라이징 작업 중...: 100%|██████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.80s/it]


## 1.4 split
preprocess/norm에 있는 노말라이즈된 데이터들을 15초 길이로 잘라서 data/train/audio에 저장

In [15]:
import subprocess

for filepath in tqdm(glob(NORM_PATH+"*.wav"), desc="음원 자르는 중..."):
    duration = librosa.get_duration(filename=filepath)
    max_last_seg_duration = 0
    sep_duration_final = 15
    sep_duration = 15

    while sep_duration > 4:
        last_seg_duration = duration % sep_duration
        if max_last_seg_duration < last_seg_duration:
            max_last_seg_duration = last_seg_duration
            sep_duration_final = sep_duration
        sep_duration -= 1

    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(args.data.train_path,"audio", f"{filename}-%04d.wav")
    subprocess.run(f'ffmpeg -i "{filepath}" -f segment -segment_time {sep_duration_final} "{out_filepath}" -y', capture_output=True, shell=True)


음원 자르는 중...:   0%|                                                                      | 0/1 [00:00<?, ?it/s]C:\Users\Polaris\AppData\Local\Temp\ipykernel_12960\1509468694.py:4: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=filepath)
음원 자르는 중...: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.63s/it]


## 1.5 무음 제거
data/train/audio에 있는 잘라진 음원들 중에 무음인 파일들을 제거

In [16]:
from sep_wav import get_ffmpeg_args
import subprocess

for filepath in tqdm(glob(args.data.train_path+"/audio/*.wav"), desc="무음 제거 중..."):
    if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

    ffmpeg_arg = get_ffmpeg_args(filepath)
    subprocess.run(ffmpeg_arg, capture_output=True, shell=True)

    start = None
    end = None

    with open(TEMP_LOG_PATH, "r", encoding="utf-8") as f:
        for line in f.readlines():
            line = line.strip()
            if "lavfi.silence_start" in line:
                start = float(line.split("=")[1])
            if "lavfi.silence_end" in line:
                end = float(line.split("=")[1])

    if start != None:
        if start == 0 and end == None:
            os.remove(filepath)
            
if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

무음 제거 중...: 100%|██████████████████████████████████████████████████████████████| 77/77 [00:06<00:00, 11.92it/s]


## 1.6 학습 데이터 중, 일부를 validaion용으로 자동으로 보내준다.
- data/train/audio에 있는 데이터 중 일정 비율만큼 알아서 data/val/audio로 이동시켜준다
    - 계산식은 다음과 같다 `max(2, min(10, 전체 데이터 * 0.01))`

In [17]:
from draw import main

main()

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.36it/s]


## 2. 데이터 전처리 (학습에 쓰기 위한)

In [20]:
from preprocess import preprocess
from ddsp.vocoder import F0_Extractor, Volume_Extractor, Units_Encoder
from diffusion.vocoder import Vocoder

# get data
sample_rate = args.data.sampling_rate
hop_size = args.data.block_size

# initialize f0 extractor
f0_extractor = F0_Extractor(
                    args.data.f0_extractor, 
                    args.data.sampling_rate, 
                    args.data.block_size, 
                    args.data.f0_min, 
                    args.data.f0_max)

# initialize volume extractor
volume_extractor = Volume_Extractor(args.data.block_size)

# initialize mel extractor
mel_extractor = None
if args.model.type == 'Diffusion':
    mel_extractor = Vocoder(args.vocoder.type, args.vocoder.ckpt, device = device)
    if mel_extractor.vocoder_sample_rate != sample_rate or mel_extractor.vocoder_hop_size != hop_size:
        mel_extractor = None
        print('Unmatch vocoder parameters, mel extraction is ignored!')

# initialize units encoder
if args.data.encoder == 'cnhubertsoftfish':
    cnhubertsoft_gate = args.data.cnhubertsoft_gate
else:
    cnhubertsoft_gate = 10             
units_encoder = Units_Encoder(
                    args.data.encoder, 
                    args.data.encoder_ckpt, 
                    args.data.encoder_sample_rate, 
                    args.data.encoder_hop_size, 
                    device = device)    

# preprocess training set
preprocess(args.data.train_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device)

# preprocess validation set
preprocess(args.data.valid_path, f0_extractor, volume_extractor, mel_extractor, units_encoder, sample_rate, hop_size, device = device)

 [Encoder Model] HuBERT Soft
 [Loading] pretrain/hubert/hubert-soft-0d54a1f4.pt
Preprocess the audio clips in : data/train\audio


  0%|                                                                                        | 0/71 [00:00<?, ?it/s]C:\ProgramData\anaconda3\envs\ddsp\lib\site-packages\torch\nn\modules\transformer.py:720: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  return torch._transformer_encoder_layer_fwd(
100%|███████████████████████████████████████████████████████████████████████████████| 71/71 [00:12<00:00,  5.83it/s]


Preprocess the audio clips in : data/val\audio


100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.65it/s]


## 3. 학습

In [ ]:
from train import ddsp_train

ddsp_train(args)

## 4. 결과물 뽑기

In [3]:
from types import SimpleNamespace
from main import inference
# configure setting
configures = {
    'model_path'            :   'exp/sins-test/model_2000.pt', # 추론에 사용하고자 하는 모델, 바로위에서 학습한 모델을 가져오면댐
    'input'                 :   'output/metero.wav', # 추론하고자 하는 노래파일의 위치 - 님들이 바꿔야댐 
    'output'                :   'output/output.wav',  # 결과물 파일의 위치
    'device'                :   'cuda',
    'spk_id'                :   '1', 
    'spk_mix_dict'          :   'None', 
    'key'                   :   '0', 
    'enhance'               :   'true' , 
    'pitch_extractor'       :   'crepe' ,
    'f0_min'                :   '50' ,
    'f0_max'                :   '1100',
    'threhold'              :   '-60',
    'enhancer_adaptive_key' :   '0'
}
cmd = SimpleNamespace(**configures)

inference(cmd)

 [DDSP Model] Sinusoids Additive Synthesiser
 [Loading] exp/sins-test/model_2000.pt


C:\ProgramData\anaconda3\envs\ddsp\lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\Polaris\Desktop\DDSP-SVC-KOR-master\ddsp\vocoder.py:472: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start set

MD5: 8b7d2087a89011d7fa8cd00f4cf15902
Pitch extractor type: crepe
Extracting the pitch curve of the input audio...


C:\ProgramData\anaconda3\envs\ddsp\lib\site-packages\torchcrepe\load.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(file, map_location=device))


Extracting the volume envelope of the input audio...
 [Encoder Model] HuBERT Soft


...

C:\Users\Polaris\Desktop\DDSP-SVC-KOR-master\ddsp\vocoder.py:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Enhancer type: nsf-hifigan
| Load HifiGAN:  pretrain/nsf_hifigan/model


C:\Users\Polaris\Desktop\DDSP-SVC-KOR-master\nsf_hifigan\models.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cp_dict = torch.load(model_path, map_location=device)


Removing weight norm...
Speaker ID: 1
Cut the input audio into 2 slices


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\ProgramData\anaconda3\envs\ddsp\lib\site-packages\torch\nn\modules\transformer.py:720: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  return torch._transformer_encoder_layer_fwd(
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [05:18<00:00, 159.12s/it]
